# 문자 단위의 번역기

##1.  데이터 전처리

1. 필요한 라이브러리 불러오기
2. 데이터 판다스로 불러오기

3. 불필요한 열 삭제하고, 훈련데이터 5만개로 줄이기

4. 디코더에 시작 토큰 \t (<sos>), 종료 토큰 \n (<eos>) 추가하기
5. Tokenizer(단어 사전, 단어장) 생성하기
6. 단어장의 크기를 변수에 담아주기(0번 토큰도 있으므로 +1하기)
7. 영어 데이터, 프랑스어 데이터들의 최대 길이 및 기본 통계 정보 구하기(패딩하기 위해)
8. 프랑스어 데이터, 즉 프랑스어 시퀀스를 2가지 버전(입력 데이터, 출력 데이터)으로 준비

**왜 프랑스어 시퀀스는 2가지 버전으로 나누어 준비?**

- 디코더의 출력과 비교할 정답 데이터로 사용 → <SOS> 토큰 필요 없음
- 교사 강요를 위해 디코더의 입력으로 사용 → <EOS> 토큰 필요 없음

1. 영어 데이터, 프랑스어 입력 데이터, 프랑스어 출력 데이터 각각에 패딩
2. 원핫 인코딩 진행
3. 훈련용 데이터, 검증용 데이터 나누어주기

In [ ]:
# 필요 라이브러리 불러오기
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

In [ ]:
# 데이터 판다스로 불러오기
import os
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5) #샘플 5개 출력

In [ ]:
# 불필요한 열 삭제하고 훈련데이터 5만개로 줄이기
lines = lines[['eng', 'fra']][:50000] # 5만개 샘플 사용
lines.sample(5)

In [ ]:
# 디코더에 시작 토큰, 종료 토큰 추가하기
sos_token = '\t'
eos_token = '\n'
lines.fra = lines.fra.apply(lambda x : '\t '+ x + ' \n')
print('전체 샘플의 수 :',len(lines))
lines.sample(5)

In [ ]:
eng_tokenizer = Tokenizer(char_level=True)   # 문자 단위로 Tokenizer를 생성합니다.
eng_tokenizer.fit_on_texts(lines.eng)               # 50000개의 행을 가진 eng의 각 행에 토큰화를 수행
input_text = eng_tokenizer.texts_to_sequences(lines.eng)    # 단어를 숫자값 인덱스로 변환하여 저장
input_text[:3]
print(input_text)

In [ ]:
fra_tokenizer = Tokenizer(char_level=True)   # 문자 단위로 Tokenizer를 생성합니다.
fra_tokenizer.fit_on_texts(lines.fra)                 # 50000개의 행을 가진 fra의 각 행에 토큰화를 수행
target_text = fra_tokenizer.texts_to_sequences(lines.fra)     # 단어를 숫자값 인덱스로 변환하여 저장
target_text[:3]

In [ ]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

In [ ]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

In [ ]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

In [ ]:
encoder_input = input_text
# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text]
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

In [ ]:
print(decoder_input[:3])
print(decoder_target[:3])

In [ ]:
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

In [ ]:
print(encoder_input[0])

In [ ]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

In [ ]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target))

## 2. 모델 훈련

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

print('⏳')

In [ ]:
# 입력 텐서 생성.
encoder_inputs = Input(shape=(None, eng_vocab_size))
# hidden size가 256인 인코더의 LSTM 셀 생성
encoder_lstm = LSTM(units = 256, return_state = True)
# 디코더로 전달할 hidden state, cell state를 리턴. encoder_outputs은 여기서는 불필요.
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# hidden state와 cell state를 다음 time step으로 전달하기 위해서 별도 저장.
encoder_states = [state_h, state_c]

In [ ]:
# 입력 텐서 생성.
decoder_inputs = Input(shape=(None, fra_vocab_size))
# hidden size가 256인 인코더의 LSTM 셀 생성
decoder_lstm = LSTM(units = 256, return_sequences = True, return_state=True)
# decoder_outputs는 모든 time step의 hidden state
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state = encoder_states)

In [ ]:
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [ ]:
from tensorflow.keras import utils

utils.plot_model(model)

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
model.summary()

In [ ]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=128, epochs=50)

## 3. 모델 테스트

# 단어 Level로 번역기 업그레이드하기
Rubric

|평가문항|상세기준|
|---|---|
1. 번역기 모델 학습에 필요한 텍스트 데이터 전처리가 잘 이루어졌다.|구두점, 대소문자, 띄어쓰기 등 번역기 모델에 요구되는 전처리가 정상적으로 진행되었다.|
|2. seq2seq 기반의 번역기 모델이 정상적으로 구동된다.|seq2seq 모델 훈련결과를 그래프로 출력해보고, validation loss그래프가 우하향하는 경향성을 보이며 학습이 진행됨이 확인되었다.|
|3. 테스트 결과 의미가 통하는 수준의 번역문이 생성되었다.|테스트용 디코더 모델이 정상적으로 만들어졌으며, input(영어)와 output(프랑스어) 모두 한글로 번역해서 결과를 출력해보았고, 둘의 내용이 유사함을 확인하였다.|


- 위 실습에서 구현한 번역기는 글자 단위(chracter-level)에서 구현된 번역기였으나, 실제로는 단어 단위(Word-level)에서 구현되는 것이 더 보편적이다.
- 진행될 프로젝트에서는 동일한 데이터셋을 사용하면서 글자 단위와는 다른 전처리와 to_categorical()함수가 아닌 임베팅 층(Embedding layer)를 추가하여 단어 단위의 번역기를 완성시키기
- 단어 단위로 할 경우, 단어의 개수가 글자 단위로 했을 경우와 비교하여 단어장의 크기도 커지고 학습 속도도 더 느려지게 된다.  따라서 학습과 테스트 시의 원활한 진행을 위해 데이터에서 상위 33,000개의 샘플만 사용하기(

In [ ]:
import tensorflow

print(tensorflow.__version__)

## Step1.정제, 정규화, 전처리(영어, 프랑스어)

## 회고

## Reference

- LSTM (https://simpling.tistory.com/19), (https://wikidocs.net/106473)
-